<a href="https://colab.research.google.com/github/tasnimlima07/Data-Analysis/blob/main/Pilot_Analysis_Employee_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connecting to Bing Query

In [ ]:
#libraries that we need
from google.cloud import bigquery
from google.colab import auth

#Authenticate
auth.authenticate_user()
#initialize the client for Bigquery


project_id = "churn-cloud-peoject"
client= bigquery.Client(project=project_id, location='Us')


In [ ]:
#Get the dataset and Table
dataset_ref = client.dataset('Employeedata', project=project_id)
dataset= client.get_dataset(dataset_ref)
table_ref= dataset.table('tbl_hr_data')
table= client.get_table(table_ref)
table.schema

In [ ]:
new_table_ref= dataset.table('tbl_new_employees')
new_table= client.get_table(new_table_ref)
new_table.schema

In [ ]:
#Convert to dataframe
df= client.list_rows(table=table).to_dataframe()
df.head()

In [ ]:
#Convert to dataframe
df2= client.list_rows(table=new_table).to_dataframe()
df2.head()

In [ ]:
!pip install pycaret

#Build Model


In [ ]:
#Get our Model
from pycaret.classification import *
setup(df, target="Quit_the_Company",
      session_id=123,
      ignore_features=['employee_id'],
      categorical_features=['Departments', 'salary'])


In [ ]:
compare_models()

In [ ]:
rf_model=create_model('rf')

In [ ]:
final_df=predict_model(rf_model)
final_df.head()

In [ ]:
final_df.info()

In [ ]:
new_prediction=predict_model(rf_model, data=df2)

In [ ]:
new_prediction.head(100)

In [ ]:
new_prediction.info()

In [ ]:
new_prediction.to_gbq('Employeedata.pilot_predictions',project_id, chunksize=None, if_exists='replace')

In [ ]:
plot_model(rf_model,plot='feature')

In [ ]:
rf_model.feature_names_in_

In [ ]:
rf_model.feature_importances_

In [ ]:
import pandas as pd
feature_table = pd.DataFrame(zip(rf_model.feature_names_in_,rf_model.feature_importances_), columns=['features','importance'])

In [ ]:
feature_table

In [ ]:
feature_table.to_gbq('Employeedata.feature_table',project_id, chunksize=None, if_exists='replace')